In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [100]:
class Tensor:
    def __init__(self, data, children = []):
        self.data = np.array(data)
        self.grad = np.zeros(self.data.shape)
        self._children = children
        self._backward = lambda: None
    
    @property
    def shape(self):
        return self.data.shape
        
    def backward(self):
        l = []
        def build(v):
            for child in v._children:
                build(child)
            l.append(v)
        build(self)
        
        topo = reversed(l)
        self.grad = 1
        for v in topo:
            v._backward()
        
    
    def __repr__(self):
        return f'Tensor([{self.data}])'
    
    def __add__(self, other):
        out = Tensor(self.data + other.data, (self, other))
    
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
            
        out._backward = _backward
        
        return out
    
    def __matmul__(self, other):
        out = Tensor(self.data @ other.data, (self, other))
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
            
        out._backward = _backward
        
        return out
    
    def __mul__(self, other):
        out = Tensor(self.data * other.data, (self, other))
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
            
        out._backward = _backward
        
        return out

In [128]:
a = Tensor([1,2,3])
b = Tensor([1,2,3])
c = Tensor([4,5,6])
d = (a + b) @ c
d.backward()

c.grad

array([2., 4., 6.])

In [129]:
import torch

a = torch.tensor([1,2,3], dtype=torch.float32, requires_grad=True)
b = torch.tensor([1,2,3], dtype=torch.float32, requires_grad=True)
c = torch.tensor([4,5,6], dtype=torch.float32, requires_grad=True)
d = (a + b) @ c
d.backward()

c.grad

tensor([2., 4., 6.])